# Sumaryzacja długich tekstów

Celem niniejszego projektu jest stworzenie sumaryzatora, który będzie w stanie dokonać sumaryzacji długich tekstów w języku angielskim (np. postów na blogach, artykułów prasowych, itp.). 
Program dokona sumaryzacji na podstawie ekstrakcji (extraction-based summarisation) lub abstrakcji (abstraction-based summarisation) w zależności od wyboru użytkownika. Gotowe podsumowanie w języku angielskim zostanie dodatkowo przełumaczone na język polski. Wyniki zostaną eksportowane do zewnętrznego pliku z rozszerzeniem .txt.

Do zrealizowania tego celu zostaną wykorzystane trzy wstępnie-przeszkolone modele NLP:

1.  Pegasus-XSum (podejście na podstawie abstrakcji)
2.  Distilbart CNN (podejście na podstawie ekstrakcji)
3.  OPUS Tatoeba English-Polish (tłumaczenie na j. polski)


In [ ]:
#!pip install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu
#!pip install sentencepiece
#!pip install transformers
#!pip install sacremoses

In [ ]:
from transformers import pipeline
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from bs4 import BeautifulSoup
import requests

# Scraping tekstu
Pobieramy interesujący nas tekst z podanego URL dzięki wykorzystaniu pakietu BeautifulSoup.

In [ ]:
def fetch_text(url):

  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  results = soup.find_all(['h1', 'p'])
  text = [result.text for result in results]
  post = ' '.join(text)

  return post

# Wstępne przetworzenie tekstu
Zescrapowany tekst dzielimy na mniejsze bloki zdań o długości do 500 znaków max.

In [ ]:
def preprocess_text(post):

  max_block = 500
  post = post.replace('.', '.<eos>')
  post = post.replace('?', '?<eos>')
  post = post.replace('!', '!<eos>')
  sentences = post.split('<eos>')
  current_block = 0 
  blocks = []
  for sentence in sentences:
      if len(blocks) == current_block + 1: 
          if len(blocks[current_block]) + len(sentence.split(' ')) <= max_block:
              blocks[current_block].extend(sentence.split(' '))
          else:
              current_block += 1
              blocks.append(sentence.split(' '))
      else:
          print(current_block)
          blocks.append(sentence.split(' '))

  for block_id in range(len(blocks)):
      blocks[block_id] = ' '.join(blocks[block_id])

  return blocks

# Tłumaczenie gotowego podsumowania na język polski

In [ ]:
def translate_to_pl(summary):

  translator_pl_to_en = pipeline("translation", model='gsarti/opus-tatoeba-eng-pol')
  translation = []

  if len(summary) >= 512:
    translation_temp = []
    parts = [summary[i:i+512] for i in range(0, len(summary), 512)]
    translation = [translator_pl_to_en(parts[i]) for i in range(0, len(parts))]
    for item in translation:
      for key in item:
        translation_temp.append(key['translation_text'])
    translation = ' '.join(translation_temp)
    translation = translation.replace(' .', '.')
    translation = translation.replace(' ,', ',')

  else:
    translation = translator_pl_to_en(summary)
    translation = translation[0]
    translation = translation['translation_text']

  return translation

# Podejście na podstawie abstrakcji

In [ ]:
def abstractive_summarization(post):
  
  tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
  model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

  blocks = preprocess_text(post)
  tokens = tokenizer(blocks, truncation=True, padding="longest", return_tensors="pt")
  result = model.generate(**tokens)

  summary = []
  for i in range(len(result)):
    summary.extend(tokenizer.decode(result[i]))
    i += 1

  summary = ''.join(summary)
  summary = summary.replace('.', '. ')

  translation = translate_to_pl(summary)

  print('EN: ' + summary + '\n')
  print('PL: ' + translation)
  save_results('abstractive_summary', summary, translation)

# Podejście na podstawie ekstrakcji

In [ ]:
def extractive_summarization(post):

  summarizer = pipeline("summarization", model = "sshleifer/distilbart-cnn-12-6")

  blocks = preprocess_text(post)
  result = summarizer(blocks, max_length=120, min_length=30, do_sample=False)
  summary = ' '.join([sentence['summary_text'] for sentence in result])
  summary = summary.replace(' .', '.')

  translation = translate_to_pl(summary)

  print('EN: ' + summary + '\n')
  print('PL: ' + translation)
  save_results('extractive_summary', summary, translation)

# Eksport wyników do zewnętrznego pliku

In [ ]:
def save_results(method, result_EN, result_PL):
  
  with open(method + '.txt', 'w') as f:
    f.write("EN: " + "\n")
    f.write(result_EN)
    f.write("\n" + "PL: " + "\n")
    f.write(result_PL)

# Sumaryzacja tekstu na przykładzie wybranego postu na blogu Hackernoon.com

In [ ]:
def summarize(url, extractive_approach=False, abstractive_approach=False):

  post = fetch_text(url)

  if extractive_approach == True:
    extractive_summarization(post)
  elif abstractive_approach == True:
    abstractive_summarization(post)
  else:
    print("ERROR: Summarization method not selected. Please choose extractive or abstractive approach.")

In [ ]:
URL = "https://hackernoon.com/what-is-the-impact-of-quantum-computing-on-blockchain-and-cryptocurrency"

In [ ]:
summarize(URL, abstractive_approach=True)

0
EN: In this article, I will be looking at the impact of quantum computing on Blockchain and cryptocurrencies. Quantum Computing has the potential to change the way we interact with the world around us. It may take a million qubits to crack 2048-bit RSA encryption, as Google claims.  With 4,099 qubits enough to crack 2048-bit RSA in ten seconds, one may shudder while thinking what a million qubit Google computer could do if introduced by 2029, as Google claims. Blockchain technology was once considered to be a distant dream. 

PL: W tym artykule, będę patrzeć na wpływ obliczeń kwantowych na Blockchain i kryptowaluta. Quantum Computing ma potencjał, aby zmienić sposób interakcji ze światem wokół nas. Może to wymagać miliona qubits, aby złamać szyfrowanie 2048-bit RSA, jak twierdzi Google. Z 4 099 Qubits wystarczająco, aby złamać 2048-bit RSA w dziesięć sekund, można drżeć, myśląc, co milion Qubit Google komputer może zrobić, jeśli wprowadzony do 2029, jak twierdzi Google. Technologia B

In [ ]:
summarize(URL, extractive_approach=True)

0
EN:  What is the Impact of Quantum Computing on Cryptocurrency?  never amongst the herd. Quantum computers have shown that they can solve the most advanced mathematical algorithms used in cryptography, the backbone of Bitcoin and cryptocurrencies. Quantum computing can also prove detrimental to today's cryptography standards with its immense speed. IBM developed one of the giant quantum computers known today in 2020 in 2020.  The global Quantum Computing market is estimated to attain $949 million by 2025, displaying a CAGR of 43% between 2020 and 2030. Quantum computers function using the properties of quantum physics for storing data and performing computations that could prove advantageous for tasks impossible for conventional computers and even state-of-the-art supercomputers. The US alone invested $625 million in multidisciplinary Quantum Information Science Research centers in 2020.  Is Bitcoin quantum-safe?  Are the current Quantum computers a threat to Bitcoin? As of date, the

Oryginalny tekst artykułu zawierał 1577 słów.

Po zastosowaniu podejścia na podstawie abstrakcji tekst został skrócony do 91 słów. Co więcej, podsumowanie składa się w większości z zupełnie nowych zdań, których oryginalnie nie sposób znaleźć w tekście źródłowym. Jest to oczekiwany rezultat przy zastosowaniu tej metody, która odzwierciedla naturalny sposób w jaki człowiek sam tworzy podsumowania.

Po zastosowaniu podejścia na podstawie ekstrakcji tekst został skrócony do 231 słów. Podsumowanie (w przeciwieństwie do podejścia abstrakcyjnego) składa się ze zdań, które znajdują się w tekście źródłowym. Te zdania, które ostatecznie trafiły do podsumowania są teoretycznie zdaniami najważniejszymi pod względem semantycznym, a zatem w teorii najlepiej ujmują główny sens całego tekstu.